# Mistral

[![Index](https://img.shields.io/badge/Index-blue)](../index.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digillia/Digillia-Colab/blob/main/tools/mistral.ipynb)

Ce bloc-note Jupyter explore les différentes capacités de [MistralAI](https://mistral.ai/) à travers son API en langage Python, et d'une manière comparable à notre exploration d'[OpenAI](./openai.ipynb). Notez que MistralAI n'offre pas encore de modèle multimodal (image, audio, video) comme OpenAI GPT4.

Docs:
- https://docs.mistral.ai/
- https://github.com/mistralai/client-python

In [1]:
import os
import sys

# Supprimer les commentaires pour installer (requirements.txt)
# !pip3 install -q -U numpy
# !pip3 install -q -U python-dotenv

# À installer dans tous les cas pour Google Colab et Github
if 'google.colab' in sys.modules or 'CI' in os.environ:
    !pip3 install mistralai

## Chargement de la Clé pour MistralAI

Il vous faut obtenir de Mistral AI une clé pour exécuter ce bloc-note Jupyter. Vous pouvez consulter [Getting Started](https://docs.mistral.ai/getting-started/quickstart/#account-setup). Ensuite, le chargement se fait soit à partir de l'environnement (fichier `.env`), soit à partir des secrets de Google Colab.

In [2]:
mistral_api_key = None
if 'google.colab' in sys.modules:
  from google.colab import userdata
  mistral_api_key = userdata.get('MISTRAL_API_KEY')
else:
  from dotenv import load_dotenv, find_dotenv
  _ = load_dotenv(find_dotenv()) # lire le fichier .env local
  mistral_api_key  = os.environ['MISTRAL_API_KEY']

In [3]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

client = MistralClient(api_key=mistral_api_key)

## Vectorisation (embeddings)

In [6]:
def get_embedding(text):
    return client.embeddings(input=[text], model='mistral-embed').data[0].embedding

emb_1 = get_embedding('Le petit chat court après la balle.')
print(emb_1)

[-0.0200042724609375, 0.01050567626953125, 0.05828857421875, 0.001552581787109375, 0.05743408203125, 0.02337646484375, 0.05865478515625, -0.010986328125, -0.032440185546875, -0.039459228515625, -0.025634765625, 0.061187744140625, -0.038818359375, -0.0208587646484375, -0.044891357421875, 0.052093505859375, 0.02130126953125, -0.0006132125854492188, -8.863210678100586e-05, -0.005443572998046875, -0.0178070068359375, -0.021026611328125, -0.0247344970703125, 0.0208282470703125, -0.0160675048828125, 0.022552490234375, -0.0222930908203125, -0.037078857421875, -0.06048583984375, 0.01039886474609375, -0.01947021484375, -0.02142333984375, -0.00684356689453125, 0.016021728515625, 0.01381683349609375, 0.007556915283203125, -0.002773284912109375, -0.003391265869140625, -0.0016632080078125, -0.013702392578125, -0.0006480216979980469, -0.055419921875, -0.021728515625, 0.0161285400390625, 0.006420135498046875, -0.03594970703125, 0.028076171875, -0.0279998779296875, 0.01070404052734375, -0.048553466796

In [7]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

emb_2 = get_embedding('Le petit chien court après la balle.')
emb_3 = get_embedding('La voiture est arrêtée au feu rouge.')

print(cosine_similarity(emb_1, emb_2))
print(cosine_similarity(emb_2, emb_3))

0.9409184709255213
0.8077965752690015


## Conversations Chat

In [8]:
completion = client.chat(
  model='mistral-small-latest',
  messages=[
    ChatMessage(role = 'system', content = 'Tu es un professeur d\'histoire.'),
    ChatMessage(role = 'user', content = 'Quand la première guerre mondiale a-t-elle commencé?'),
    ChatMessage(role = 'assistant', content = 'La première guerre mondiale a commencé en 1914.'),
    ChatMessage(role = 'user', content = 'Quand a-t-elle fini?')
  ]
)

response = completion.choices[0].message.content
print(response)

En effet, je suis un assistant virtuel capable de fournir des informations sur une variété de sujets, y compris l'histoire. La Première Guerre mondiale s'est terminée en 1918, plus précisément le 11 novembre. C'est pourquoi cette date est commémorée chaque année comme l'Armistice Day ou le Veterans Day dans certains pays.
